# Final_Year_Project_in_AASTU_2024

## Residual Neural Network architectures 

In [40]:
import cv2
import os
import pickle
import numpy as np
import pandas as pd
from glob import glob
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

import matplotlib.pyplot as plt

In [41]:
class MasterImage(object):

    def __init__(self, PATH='', IMAGE_SIZE=50):
        self.PATH = PATH
        self.IMAGE_SIZE = IMAGE_SIZE
        self.image_data = []
        self.x_data = []
        self.y_data = []
        self.CATEGORIES = []
        self.list_categories = []

    def get_categories(self):
        for path in os.listdir(self.PATH):
            if '.DS_Store' in path:
                pass
            else:
                self.list_categories.append(path)
        print("Found Categories:", self.list_categories)
        return self.list_categories
    
    def process_image(self):
        try:
            self.CATEGORIES = self.get_categories()
            for category in self.CATEGORIES:
                train_folder_path = os.path.join(self.PATH, category)
                class_index = self.CATEGORIES.index(category)
                for img in os.listdir(train_folder_path):
                    image_path = os.path.join(train_folder_path, img)
                    try:
                        image_data_temp = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                        image_temp_resize = cv2.resize(image_data_temp, (self.IMAGE_SIZE, self.IMAGE_SIZE))
                        self.image_data.append((image_temp_resize, class_index))
                    except:
                        pass
            self.image_data = [data for data in self.image_data if data[0] is not None]

            X_Data = []
            Y_Data = []
            for img, label in self.image_data:
                X_Data.append(img)
                Y_Data.append(label)

            X_Data = np.asarray(X_Data) / 255.0
            Y_Data = np.asarray(Y_Data)
            X_Data = X_Data.reshape(-1, self.IMAGE_SIZE, self.IMAGE_SIZE, 1)

            return X_Data, Y_Data
        except Exception as e:
            print("Failed to run Function process_image:", e)

    def pickle_image(self):
        try:
            X_Data, Y_Data = self.process_image()

            with open('X_Data.pickle', 'wb') as pickle_out:
                pickle.dump(X_Data, pickle_out)

            with open('Y_Data.pickle', 'wb') as pickle_out:
                pickle.dump(Y_Data, pickle_out)

            print("Pickle files created successfully.")
            return X_Data, Y_Data
        except Exception as e:
            print("Failed to create pickle files:", e)

    def load_dataset(self):
        try:
            if os.path.exists('X_Data.pickle') and os.path.exists('Y_Data.pickle'):
                with open('X_Data.pickle', 'rb') as pickle_in:
                    X_Data = pickle.load(pickle_in)

                with open('Y_Data.pickle', 'rb') as pickle_in:
                    Y_Data = pickle.load(pickle_in)

                print('Dataset loaded from pickle files.')
                return X_Data, Y_Data
            else:
                print('Pickle files not found.')
                print('Creating pickle files and loading the dataset...')
                X_Data, Y_Data = self.pickle_image()
                return X_Data, Y_Data
        except Exception as e:
            print("Failed to load dataset:", e)



In [42]:
path = 'ISIC-images'
a = MasterImage(PATH=path, IMAGE_SIZE=80)
X_Data, Y_Data = a.load_dataset()

Dataset loaded from pickle files.


In [43]:
# Split the scaled data into train, validation, and test sets
X_train , X_test , y_train , y_test  = train_test_split(X_Data, Y_Data, test_size=0.2, random_state=42)
X_train , X_val , y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

# Print the shape of each dataset
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (7032, 80, 80, 1)
Shape of y_train: (7032, 8)
Shape of X_val: (2344, 80, 80, 1)
Shape of y_val: (2344, 8)
Shape of X_test: (2344, 80, 80, 1)
Shape of y_test: (2344, 8)


In [44]:


# Define the Residual Block
def residual_block(x, filters, downsample=False):
    shortcut = x

    # First convolutional layer
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=(1 if not downsample else 2, 1 if not downsample else 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second convolutional layer
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Downsample the shortcut if needed
    if downsample:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=(2, 2), padding='valid')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    # Add the shortcut to the output
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

# Define the ResNet model
def create_resnet_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Initial convolutional layer
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, downsample=True)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, downsample=True)
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, downsample=True)
    x = residual_block(x, filters=512)

    # Global average pooling and output layer
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create the ResNet model
input_shape = (80, 80, 1)
num_classes = 8
model = create_resnet_model(input_shape, num_classes)
model.summary()

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)



Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 80, 80, 1)]          0         []                            
                                                                                                  
 conv2d_135 (Conv2D)         (None, 40, 40, 64)           3200      ['input_8[0][0]']             
                                                                                                  
 batch_normalization_125 (B  (None, 40, 40, 64)           256       ['conv2d_135[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 re_lu_34 (ReLU)             (None, 40, 40, 64)           0         ['batch_normalization_12

                                                                                                  
 conv2d_144 (Conv2D)         (None, 20, 20, 128)          147584    ['re_lu_41[0][0]']            
                                                                                                  
 batch_normalization_134 (B  (None, 20, 20, 128)          512       ['conv2d_144[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 add_59 (Add)                (None, 20, 20, 128)          0         ['batch_normalization_134[0][0
                                                                    ]',                           
                                                                     're_lu_40[0][0]']            
                                                                                                  
 re_lu_42 

 batch_normalization_143 (B  (None, 5, 5, 512)            2048      ['conv2d_153[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 re_lu_49 (ReLU)             (None, 5, 5, 512)            0         ['batch_normalization_143[0][0
                                                                    ]']                           
                                                                                                  
 conv2d_154 (Conv2D)         (None, 5, 5, 512)            2359808   ['re_lu_49[0][0]']            
                                                                                                  
 batch_normalization_144 (B  (None, 5, 5, 512)            2048      ['conv2d_154[0][0]']          
 atchNormalization)                                                                               
          

In [45]:
# Train the model
batch_size = 32
epochs = 15

model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val)
)


Epoch 1/15
220/220 [==============================] - 292s 1s/step - loss: 1.1041 - accuracy: 0.6496 - val_loss: 1.8116 - val_accuracy: 0.6523
Epoch 2/15
220/220 [==============================] - 300s 1s/step - loss: 0.9631 - accuracy: 0.6842 - val_loss: 1.7608 - val_accuracy: 0.6527
Epoch 3/15
220/220 [==============================] - 297s 1s/step - loss: 0.9220 - accuracy: 0.6958 - val_loss: 1.4143 - val_accuracy: 0.6378
Epoch 4/15
220/220 [==============================] - 288s 1s/step - loss: 0.8955 - accuracy: 0.6962 - val_loss: 1.1318 - val_accuracy: 0.6536
Epoch 5/15
220/220 [==============================] - 288s 1s/step - loss: 0.8634 - accuracy: 0.7069 - val_loss: 1.6267 - val_accuracy: 0.6523
Epoch 6/15
220/220 [==============================] - 286s 1s/step - loss: 0.8419 - accuracy: 0.7125 - val_loss: 1.3077 - val_accuracy: 0.6335
Epoch 7/15
220/220 [==============================] - 287s 1s/step - loss: 0.7938 - accuracy: 0.7324 - val_loss: 1.8045 - val_accuracy: 0.6527

In [46]:

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

74/74 [==============================] - 22s 292ms/step - loss: 9.1871 - accuracy: 0.6608
Test Loss: 9.1871
Test Accuracy: 0.6608
